In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lightgbm import train
from sklearn.model_selection import train_test_split 
from scipy.stats import beta, bernoulli

In [22]:
df=pd.read_csv('../../data/preprocessed_data.csv')
df.head()

,fever,headache,muscle pain,joint pain,rash,nausea,vomiting,eye pain,abdominal pain,lymphadenopathy,chills,diarrhea,fatigue,dengue
0,0,1,0,0,1,1,0,0,1,1,1,0,1,1
1,1,0,1,1,1,1,0,1,0,1,0,0,1,0
2,1,0,0,0,0,1,1,0,1,0,1,0,1,0
3,1,0,0,1,0,1,0,1,1,1,0,1,0,0
4,1,0,0,0,1,0,1,1,1,1,0,1,0,1


In [23]:
for colum in df.columns:
    print(df[colum].value_counts())

fever
1    2817
0    1569
Name: count, dtype: int64
headache
0    2213
1    2173
Name: count, dtype: int64
muscle pain
0    2679
1    1707
Name: count, dtype: int64
joint pain
0    2457
1    1929
Name: count, dtype: int64
rash
0    2811
1    1575
Name: count, dtype: int64
nausea
0    2537
1    1849
Name: count, dtype: int64
vomiting
0    2417
1    1969
Name: count, dtype: int64
eye pain
0    2387
1    1999
Name: count, dtype: int64
abdominal pain
0    2741
1    1645
Name: count, dtype: int64
lymphadenopathy
0    2574
1    1812
Name: count, dtype: int64
chills
1    2473
0    1913
Name: count, dtype: int64
diarrhea
0    2871
1    1515
Name: count, dtype: int64
fatigue
0    2880
1    1506
Name: count, dtype: int64
dengue
0    2472
1    1914
Name: count, dtype: int64


In [24]:
# ベータ分布に基づいてバイナリ特徴量の生成を行う
# ベータ分布のパラメータ alpha, beta を決定するため、各特徴量の0,1の出現頻度を計算
n_samples = 5000  # 生成するサンプルの数
generated_data = []

for col in df.columns:
    positive_count = df[col].sum()  # 1の数
    negative_count = len(df[col]) - positive_count  # 0の数

    # ベータ分布のパラメータを計算 (ラプラススムージング: +1)
    alpha = positive_count + 1
    beta_param = negative_count + 1

    # ベータ分布に基づいてサンプルを生成 (1が出る確率を推定)
    probability = beta.rvs(alpha, beta_param)
    
    # 推定された確率に基づいてベルヌーイ分布から新しいサンプルを生成
    generated_feature = bernoulli.rvs(probability, size=n_samples)
    generated_data.append(generated_feature)

# 生成されたデータをDataFrameに変換
generated_df = pd.DataFrame(np.array(generated_data).T, columns=df.columns)

In [25]:
for colum in generated_df.columns:
    print(generated_df[colum].value_counts())

fever
1    3198
0    1802
Name: count, dtype: int64
headache
0    2531
1    2469
Name: count, dtype: int64
muscle pain
0    3025
1    1975
Name: count, dtype: int64
joint pain
0    2836
1    2164
Name: count, dtype: int64
rash
0    3332
1    1668
Name: count, dtype: int64
nausea
0    2925
1    2075
Name: count, dtype: int64
vomiting
0    2670
1    2330
Name: count, dtype: int64
eye pain
0    2694
1    2306
Name: count, dtype: int64
abdominal pain
0    3096
1    1904
Name: count, dtype: int64
lymphadenopathy
0    2911
1    2089
Name: count, dtype: int64
chills
1    2766
0    2234
Name: count, dtype: int64
diarrhea
0    3263
1    1737
Name: count, dtype: int64
fatigue
0    3289
1    1711
Name: count, dtype: int64
dengue
0    2741
1    2259
Name: count, dtype: int64


In [26]:
data_count=len(df)+len(generated_df)
data_count

9386

In [27]:
#trainデータ
train=df[:int(len(df)*0.6)]
train=pd.concat([train,generated_df],axis=0, ignore_index=True)

#testデータ
test=df[int(len(df)*0.6):]

In [28]:
len(test)/len(train)

0.22998296422487224

In [29]:
train.to_csv('../../data/train.csv',index=False)
test.to_csv('../../data/test.csv',index=False)

In [33]:
df['dengue'].value_counts()

dengue
0    2472
1    1914
Name: count, dtype: int64

In [34]:
train['dengue'].value_counts()

dengue
0    4056
1    3575
Name: count, dtype: int64